In [82]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [77]:
import scanpy as sc
import anndata
from SpaceFlow import SpaceFlow
import matplotlib.pyplot as plt
import umap
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
import torch

from src.visualization import plot_embeddings

In [87]:
DATASET = "Breast Cancer"

if DATASET == "Breast Cancer":
    ADATA_PATH = 'data/V1_Breast_Cancer_Block_A_Section_1/breast_cancer.h5ad'
    GT_PATH = 'data/V1_Breast_Cancer_Block_A_Section_1/cpdb_scores/thresholded_interaction_matrix.tsv'
elif DATASET == "Lymph Node":
    ADATA_PATH = 'data/V1_Human_Lymph_Node/lymph_node.h5ad'
    GT_PATH = 'data/V1_Human_Lymph_Node/cpdb_scores/thresholded_interaction_matrix.tsv'

# Load data and get embeddings

In [88]:
adata = anndata.read_h5ad(ADATA_PATH)
cell_types = adata.obs["leiden"].values.astype(int)
adata.X = adata.raw.X
adata

AnnData object with n_obs × n_vars = 3001 × 22240
    obs: 'in_tissue', 'array_row', 'array_col', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'hvg', 'leiden', 'leiden_colors', 'log1p', 'neighbors', 'pca', 'spatial', 'umap'
    obsm: 'X_pca', 'X_umap', 'spatial'
    varm: 'PCs'
    layers: 'log_norm'
    obsp: 'connectivities', 'distances'

In [89]:
# Load the ground truth data
ground_truth = pd.read_csv(GT_PATH, sep='\t', index_col=0).values.astype(int)

In [90]:
sf = SpaceFlow.SpaceFlow(adata=adata)
sf.preprocessing_data(n_top_genes=2000)

In [91]:
sf_embeddings = []
for seed in [42, 13, 21]:
    print(f"Training with seed {seed}")
    embedding = sf.train(
        spatial_regularization_strength=0.1,
        z_dim=64, 
        lr=1e-3, 
        epochs=500, 
        max_patience=50, 
        min_stop=100, 
        random_seed=seed,
        gpu=0, 
        regularization_acceleration=True, 
        edge_subset_sz=1000000
    )
    sf_embeddings.append(embedding)

Training with seed 42
Epoch 2/500, Loss: 1.4730812311172485
Epoch 12/500, Loss: 1.4321227073669434
Epoch 22/500, Loss: 1.3491175174713135
Epoch 32/500, Loss: 1.1115518808364868
Epoch 42/500, Loss: 0.7639027237892151
Epoch 52/500, Loss: 0.4779401421546936
Epoch 62/500, Loss: 0.25889357924461365
Epoch 72/500, Loss: 0.16981270909309387
Epoch 82/500, Loss: 0.12061193585395813
Epoch 92/500, Loss: 0.09782376885414124
Epoch 102/500, Loss: 0.07699981331825256
Epoch 112/500, Loss: 0.06231538951396942
Epoch 122/500, Loss: 0.06220690533518791
Epoch 132/500, Loss: 0.0733335092663765
Epoch 142/500, Loss: 0.05906536430120468
Epoch 152/500, Loss: 0.06884428858757019
Epoch 162/500, Loss: 0.0596904382109642
Epoch 172/500, Loss: 0.06491965055465698
Epoch 182/500, Loss: 0.047547124326229095
Epoch 192/500, Loss: 0.04776524007320404
Epoch 202/500, Loss: 0.04975150525569916
Epoch 212/500, Loss: 0.04659229889512062
Epoch 222/500, Loss: 0.047969382256269455
Epoch 232/500, Loss: 0.05032311752438545
Epoch 242/5

In [92]:
sf_embeddings[0].shape

(3001, 64)

In [93]:
# Convert cell_types to one-hot tensor
num_classes = len(np.unique(cell_types))
cell_types_one_hot = np.eye(num_classes)[cell_types]

plot_embeddings(torch.from_numpy(sf_embeddings[0]), torch.from_numpy(cell_types_one_hot), title=f"SpaceFlow Embeddings ({DATASET})", save_path=f"plots/{DATASET}_spaceflow_embeddings.png")

/opt/anaconda3/envs/spaceflow/lib/python3.8/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


## Predict interactions using embeddings

In [85]:
num_samples = 11000

accuracies = []
f1_scores = []

for run, embeddings in enumerate(sf_embeddings):
    embedding_pairs = []
    interaction_labels = []

    indices = np.random.randint(0, embeddings.shape[0], (num_samples, 2))

    embedding_pairs = []
    interaction_labels = []

    for idx in indices:
        i, j = idx
        cell_type_i = cell_types[i]
        cell_type_j = cell_types[j]
        interaction = ground_truth[cell_type_i, cell_type_j]
        
        concatenated_embedding = np.concatenate((embeddings[i], embeddings[j]))
        
        embedding_pairs.append(concatenated_embedding)
        interaction_labels.append(interaction)

    embedding_pairs = np.stack(embedding_pairs)
    interaction_labels = np.array(interaction_labels)

    # Create train/test split
    train_pairs, test_pairs, train_labels, test_labels = train_test_split(
        embedding_pairs, interaction_labels, test_size=(1./11), random_state=42
    )

    # Flatten the embeddings for logistic regression
    train_pairs_flat = train_pairs.reshape(train_pairs.shape[0], -1)
    test_pairs_flat = test_pairs.reshape(test_pairs.shape[0], -1)

    # Initialize the logistic regression model
    log_reg = LogisticRegression(max_iter=1000)

    # Train the model
    log_reg.fit(train_pairs_flat, train_labels)

    # Predict on the test set
    test_predictions = log_reg.predict(test_pairs_flat)

    # Calculate accuracy
    accuracy = accuracy_score(test_labels, test_predictions)
    f1 = f1_score(test_labels, test_predictions)

    print(f"Run {run} | Test Accuracy: {accuracy:.4f} | Test F1 Score: {f1:.4f}")

    accuracies.append(accuracy)
    f1_scores.append(f1)

print(f"Average Accuracy: {np.mean(accuracies):.4f} ± {np.std(accuracies):.4f}")
print(f"Average F1 Score: {np.mean(f1_scores):.4f} ± {np.std(f1_scores):.4f}")

Run 0 | Test Accuracy: 0.7320 | Test F1 Score: 0.7428
Run 1 | Test Accuracy: 0.7690 | Test F1 Score: 0.7755
Run 2 | Test Accuracy: 0.7910 | Test F1 Score: 0.8059
Average Accuracy: 0.7640 ± 0.0243
Average F1 Score: 0.7748 ± 0.0258
